### Convert RGBA Array to Bitmap Image (Spectrogram)

In [1]:
def array_to_bitmap(rgba_array, out_basename, folder):

    if rgba_array.ndim != 3 or rgba_array.shape[2] != 4:
        raise ValueError("Expected an array for RGBA data")

    # Convert to uint8 
    rgba_uint8 = (rgba_array * 255).astype(np.uint8)
    
    #Directory for spectrograms
    path = os.path.join(folder, 'spectrograms')
    os.makedirs(path, exist_ok=True)  
    bmp_filename = os.path.join(path, f"{out_basename}.bmp")

    # Create image and save as bitmap
    #bmp_filename = f"{out_basename}.bmp"
    img = Image.fromarray(rgba_uint8, mode="RGBA")
    img.save(bmp_filename)


### Divide Large IQ Sample Input into Spectrogram Dictated Sample Counts

In [2]:
def IQsample_load(file, c):
    # Number of spectrograms to be generated
    num_spectrograms = math.floor(os.path.getsize(file) / (c * 8))
    
    dirname, basename = os.path.split(file)
    folder = dirname.split('-')
    
    #Create folder name based on file name
    if folder[0] == 'Post':
        prefix = 'post'
    elif folder[0] == 'Pre':
        prefix = 'pre'
    else:
        raise ValueError(f"Unknown channel prexif")
        
    channel_type = folder[2]
    
    # Folder name based on c
    if c == 33024:
        folder_num = '256x256'
    elif c == 65664:
        folder_num = '256x512'
    elif c == 131328:
        folder_num = '512x512'
    elif c == 262400:
        folder_num = '512x1024'
    elif c == 524800:
        folder_num = '1024x1024'
    else:
        raise ValueError(f"Unknown c value: {c}")

    # Ensure folder exists
    folder_name = folder_num + "-" + channel_type
    os.makedirs(folder_name, exist_ok=True)

    # Read and save samples
    offset = 0
    for counter in range(num_spectrograms):
        # Read one block of IQ samples
        iq_sample = np.fromfile(file, dtype=np.complex64, count=c, offset=offset)

        # Save it to its own binary file
        out_path = os.path.join(folder_name, f'{prefix}-sample-{counter}.bin')
        iq_sample.tofile(out_path)

        # Update offset (8 bytes per complex64 sample)
        offset += c * 8

Spectrogram SciPy:

* spectrogram(time_domain_input, sampling_freq, nsperg, noverlap, return_onesided)
    * nperseg = # of samples per segment
    * noverlap = number of samples to overlap between segments
    * return_onesided = True (returns a two-sided spectrogram with real and complex)

| Spectrogram Dimensions | nperseg | noverlap | Samples Needed|
|:------:|:------:|:------:|:------:|
| 256x256 | 256 | 128 | 33,024 |
| 256x512 | 256 | 128 | 65,664 |
| 512x512 | 512 | 256 | 131,328 |
| 512x1024 | 512 | 256 | 262,400 |
| 1024x1024 | 1024 | 512 | 524,800 |

In [3]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import spectrogram
from PIL import Image
import math
import os

# === CONFIG ===
post_filename = "Post-Channel-AWGN/post_channel.iq"
pre_filename = "Pre-Channel-AWGN/pre_channel.iq"
fs = 23.04e6

# Create individual Pre-Channel IQ files for each spectrogram
IQsample_load(pre_filename, 65664)
IQsample_load(pre_filename, 131328)
IQsample_load(pre_filename, 262400)
IQsample_load(pre_filename, 524800)

# Create individual Post-Channel IQ files for each spectrogram
IQsample_load(post_filename, 65664)
IQsample_load(post_filename, 131328)
IQsample_load(post_filename, 262400)
IQsample_load(post_filename, 524800)

In [17]:
directory_path = "1024x1024-AWGN" # Image size directory 
param = directory_path.split("-")

#Gets parameter values based on image size
if param[0] == '256x256' or param[0] == '256x512':
    nper = 256 #nperseg
    nov = 128 #noverlap
elif param[0] == '512x512' or param[0] == '512x1024':
    nper = 512 #nperseg
    nov = 256 #noverlap
elif param[0] == '1024x1024':
    nper = 1024 #nperseg
    nov = 512 #noverlap
else:
    raise ValueError(f"Unknown image size in directory path")


In [18]:
# === COMPUTE SPECTROGRAM ===
sample_num = 0
pre = "pre"
post = "post"

files = [
    f for f in os.listdir(directory_path)
    if os.path.isfile(os.path.join(directory_path, f))
]

# plots a spectrogram for each iq sample in the given folder
for sample_num in enumerate(files):   
    
    pre_filepath = os.path.join(directory_path,f'{pre}-sample-{sample_num[0]}.bin')
    post_filepath = os.path.join(directory_path,f'{post}-sample-{sample_num[0]}.bin')
    
    pre_iq = np.fromfile(pre_filepath, dtype=np.complex64, count=262656)
    post_iq = np.fromfile(post_filepath, dtype=np.complex64, count=262656)
    
    # Smag is power spectral density (amplitude squared)   
    f_pre, t_pre, pre_Smag = spectrogram(pre_iq, fs=fs, nperseg=nper, noverlap=nov, return_onesided=False)
    f_post, t_post, post_Smag = spectrogram(post_iq, fs=fs, nperseg=nper, noverlap=nov, return_onesided=False)

    #print(f_pre.size)
    #print(t_pre.size)
    #print(pre_Smag.size)
    #print(post_Smag.size)
    # Convert power to power in decibels
    pre_Smag_dB = 10 * np.log10(pre_Smag + 1e-12) # +1e-12 prevents infinite log when spectrogram power = 0
    post_Smag_dB = 10 * np.log10(post_Smag + 1e-12)

    # min and max values mapped to endpoints of colormap
    # 0db = full scale power, weak components -100db or less
    vmin, vmax = -120, 0 
    norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)
    color_map = cm.get_cmap('viridis')
    # Map normalized data (RGBA floats in [R, G, B, A] values between 0-1)
    rgba_pre = color_map(norm(pre_Smag_dB))
    rgba_post = color_map(norm(post_Smag_dB))

    # Create a bitmap image from these RGBA values for pre and post-channel spectrograms
    array_to_bitmap(rgba_pre, f'spectrogram_pre_{sample_num[0]}', directory_path)
    array_to_bitmap(rgba_post, f'spectrogram_post_{sample_num[0]}', directory_path)
    
    # === PLOT CLEAN SPECTROGRAM ===
    height_px = 1024
    width_px = 512
    plt.figure(figsize=(6, 4), frameon=False)
    plt.axis("off")             # remove axes
    plt.imshow(pre_Smag_dB, aspect='auto', origin='lower', cmap='viridis')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # remove all padding
    plt.margins(0, 0)
    pre_png_file = os.path.join(directory_path, 'spectrograms', f'spectrogram_pre_{sample_num[0]}.png')
    plt.savefig(pre_png_file, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

    plt.figure(figsize=(6, 4), frameon=False)
    plt.axis("off")             # remove axes
    plt.imshow(post_Smag_dB, aspect='auto', origin='lower', cmap='viridis')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # remove all padding
    plt.margins(0, 0)
    post_png_file = os.path.join(directory_path, 'spectrograms', f'spectrogram_post_{sample_num[0]}.png')
    plt.savefig(post_png_file, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: '1024x1024-AWGN/pre-sample-1.bin'